In [ ]:
# default_exp linkage

# Linkage analysis pipeline

This pipeline is using paramlink2 to do linkage analysis. The R code is bridged to python through rpy2. It run linkage analysis from batch to batch. Its input is the intermediate result of seqlink.
Next, I will make it to a sos pipeline. Run all the chromosomes in parallel.

In [ ]:
#export
import numpy as np
import pandas as pd
import pickle
from itertools import repeat
import numbers

#Import necessary packages
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
base = importr('base')
base.options(expressions = 5e5)
#Must be activated
pandas2ri.activate()
paramlink2=importr('paramlink2')
pedprobr=importr('pedprobr')
pedtools = importr('pedtools')

import time
from concurrent.futures import ProcessPoolExecutor,ThreadPoolExecutor

## Functions to deal with haplotypes

In [ ]:
#export
def get_allele(s):
    a = s[1] if s[0].isupper() else s[0]
    return 0 if a=='?' else int(a)

def name_haps(snps):
    name = []
    for i in snps:
        name += [i+'_A0',i+'_A1']
    return name

def get_fam_hap(haps,variants,vcf=None):
    new_haps,new_iid = [],[]
    iid = haps[:,1]
    haps = haps[:,2:]
    for i in range(0,haps.shape[0],2):
        cur_iid=iid[i]
        new_iid.append(cur_iid)
        if vcf is None or vcf[cur_iid]:#have vcf
            hap_a01 = []
            for a0,a1 in zip(haps[i],haps[i+1]): #loop through variants
                hap_a01 += [get_allele(a0),get_allele(a1)]
        else:
            hap_a01 = [0,0]*haps.shape[1] #set missing vcf to 0
        new_haps.append(hap_a01)
    new_haps = pd.DataFrame(new_haps)
    new_haps.index = new_iid
    new_haps.columns = name_haps(variants)
    #remove variants with only 1 or 2 as alleles, return None
    idx=[]
    for i in range(0,new_haps.shape[1],2):
        v=set(new_haps.iloc[:,i]).union(set(new_haps.iloc[:,i+1]))
        if 1 not in v or 2 not in v:
            idx.append(False)
        else:
            idx.append(True)
    if sum(idx)==0:
        return None
    return new_haps.loc[:,np.repeat(np.array(idx),2)],idx

def get_fam_geno(haps,variants,vcf=None):
    new_haps,new_iid = [],[]
    iid = haps[:,1]
    haps = haps[:,5:]
    for i in range(haps.shape[0]):
        cur_iid=iid[i]
        new_iid.append(cur_iid)
        if vcf is None or vcf[cur_iid]:#have vcf
            hap_a01 = []
            for a01 in haps[i]: #loop through variants
                hap_a01 += [int(a) for a in a01]
        else:
            hap_a01 = [0,0]*haps.shape[1] #set missing vcf to 0
        new_haps.append(hap_a01)
    new_haps = pd.DataFrame(new_haps)
    new_haps.index = new_iid
    new_haps.columns = name_haps(variants)
    #remove variants with only 1 or 2 as alleles, return None
    idx=[]
    for i in range(0,new_haps.shape[1],2):
        v=set(new_haps.iloc[:,i]).union(set(new_haps.iloc[:,i+1]))
        if 1 not in v or 2 not in v:
            idx.append(False)
        else:
            idx.append(True)
    if sum(idx)==0:
        return None
    return new_haps.loc[:,np.repeat(np.array(idx),2)],idx

## All genes from haps to peds

#### compare multithread and multiprocess

In [ ]:
#export
def format_haps_bunch(dhaps,fam,vcfs=None,cutoff=None,haplotype=True):
    gene_variants = {}
    gene_haps = {}
    for g in dhaps.keys():
        haps = dhaps[g]['predata']
        with ProcessPoolExecutor(max_workers = 10) as executor:
            if haplotype:
                results = executor.map(get_fam_hap,[haps[k][2] for k in haps.keys()],[haps[k][0] for k in haps.keys()],[vcfs[k] if vcfs else None for k in haps.keys()])
            else:
                results = executor.map(get_fam_geno,[haps[k][2] for k in haps.keys()],[haps[k][0] for k in haps.keys()],[vcfs[k] if vcfs else None for k in haps.keys()])
        for f,hap in  zip(haps.keys(),results):
            if hap is None: #remove only have 1 or 2 variants
                continue
            if f not in gene_variants.keys():
                gene_variants[f] = {'genes':[],'variants':[],'freqs':[]}
                gene_haps[f] = hap[0]
            else:
                gene_haps[f] = pd.concat([gene_haps[f],hap[0]],axis=1)
            idx=hap[1] #False for variants only have 1 or 2.
            gene_variants[f]['genes'] += [g]*sum(idx)
            gene_variants[f]['variants'] += list(haps[f][0][idx])
            gene_variants[f]['freqs'] += list(haps[f][1][idx])
    for i,j in gene_variants.items():
        j=pd.DataFrame(j)
        if cutoff is not None:
            frq_idx=np.array(j['freqs'])>cutoff
            j=j.loc[frq_idx,:]
            gene_haps[i]=gene_haps[i].loc[:,np.repeat(frq_idx,2)] 
        redup_idx = ~gene_haps[i].columns.duplicated()
        gene_haps[i] = pd.concat([fam[i],gene_haps[i].iloc[:,redup_idx]],axis=1)
        j['uniq'] = list(redup_idx[range(0,len(redup_idx),2)])
        gene_variants[i] = j
    return gene_variants,gene_haps



def calculate_ped_lod(ped,afreq=None,rho=0,model = "AD",chrom = "AUTOSOMAL",penetrances = [0.01,0.9,0.9],dfreq=0.001):
    aff=ped.iloc[:,5]
    mped = pedtools.as_ped(ped.drop(ped.columns[5], axis=1),famid_col = 1,id_col = 2,fid_col = 3,mid_col = 4,sex_col = 5)
    if afreq is not None:
        mped = pedtools.setLocusAttributes(mped,locusAttributes=[base.list(afreq=base.c(1-i,i)) for i in afreq])
    modAD = paramlink2.diseaseModel(model,chrom,pd.Series(penetrances),dfreq)
    if isinstance(rho,numbers.Number):
        res = _calculate_ped_lod(mped, aff = aff, model = modAD,rho=rho)
    else:
        res=None
        for r in rho:
            tmp = _calculate_ped_lod(mped, aff = aff, model = modAD,rho=r)
            if res is None:
                res=tmp
                res.columns = ['MARKER','LOD'+str(round(r,2))]
            else:
                res['LOD'+str(round(r,2))]=tmp.LOD
        res.index=list(res.MARKER)
        res=res.iloc[:,1:]
    return res

def _calculate_ped_lod(mped, aff, model,rho):
    res = paramlink2.lod(mped, aff, model,rho)
    try:
        res = pd.DataFrame(res)[['MARKER','LOD']]
    except:
        res = pd.DataFrame([[ped.columns[6],res[0]]],columns=['MARKER','LOD'])
    return res

def parallel_lods(haps,afreqs=None,rho=0):
    start = time.perf_counter()
    if afreqs is None:
        with ProcessPoolExecutor(max_workers = 10) as executor:
            results = executor.map(calculate_ped_lod,haps.values(),repeat(rho))
    else:
        with ProcessPoolExecutor(max_workers = 10) as executor:
            results = executor.map(calculate_ped_lod,haps.values(),afreqs,repeat(rho))
    print(time.perf_counter()-start)
    return {k:res for k,res in zip(haps.keys(),results)}

def sum_variant_lods(lods):
    variants = {}
    for lod in lods:
        for m,l in zip(lod['MARKER'],lod['LOD']):
            if m in variants.keys():
                variants[m] += l
            else:
                variants[m] = l
    var_lst = []
    for var,lod in variants.items():
        snp = var[:-3]
        var_lst.append(snp.split(':')+[snp,lod])
    variants=pd.DataFrame(var_lst,columns=['CHR','POS','A0','A1','SNP','LOD'])
    variants.POS = variants.POS.astype(int)
    variants.sort_values('POS')
    return variants

## Testing

In [ ]:
import pandas as pd
import numpy as np
import pickle
from SEQLinkage.linkage import *

### Read fam

In [ ]:
fam17 = pd.read_csv('../data/new_trim_ped_famless17_no:xx.fam',delim_whitespace=True,header=None,names=['fid','iid','fathid','mothid','sex','ad'])
fam17.index = list(fam17.iid)
fam17.ad[fam17.ad==-9]=0
fam17_d = {}
for i in fam17.fid.unique():
    fam17_d[i] = fam17[fam17.fid==i]

/tmp/2440590.1.high_mem.q/ipykernel_20253/2396333578.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fam17.ad[fam17.ad==-9]=0


## Read haplotypes

In [ ]:
import glob

In [ ]:
glob.glob('../data/wg20220316/chr9test/tmp/CACHE/chr9test*.pickle')[-3:]

['../data/wg20220316/chr9test/tmp/CACHE/chr9test2.pickle',
 '../data/wg20220316/chr9test/tmp/CACHE/chr9test1.pickle',
 '../data/wg20220316/chr9test/tmp/CACHE/chr9test0.pickle']

In [ ]:
for i in glob.glob('../data/wg20220316/chr9test/tmp/CACHE/chr9test*.pickle')[-3:]:
    print(i)
    run_gene_lods(i[:-7],fam17_d,cutoff=0.05)

../data/wg20220316/chr9test/tmp/CACHE/chr9test2.pickle


In [ ]:
for i in glob.glob('../data/wg20220316/chr10test/tmp/CACHE/chr10test*.pickle'):
    print(i)
    run_gene_lods(i[:-7],fam17_d)

../data/wg20220316/chr10test/tmp/CACHE/chr10test43.pickle
48.879359588027
../data/wg20220316/chr10test/tmp/CACHE/chr10test42.pickle
378.9136514291167
../data/wg20220316/chr10test/tmp/CACHE/chr10test41.pickle
780.737620703876
../data/wg20220316/chr10test/tmp/CACHE/chr10test40.pickle
716.6588892191648
../data/wg20220316/chr10test/tmp/CACHE/chr10test39.pickle
461.18784821778536
../data/wg20220316/chr10test/tmp/CACHE/chr10test38.pickle
594.9520736783743
../data/wg20220316/chr10test/tmp/CACHE/chr10test37.pickle
814.8921134918928
../data/wg20220316/chr10test/tmp/CACHE/chr10test36.pickle
673.1356900706887
../data/wg20220316/chr10test/tmp/CACHE/chr10test35.pickle
703.0796470716596
../data/wg20220316/chr10test/tmp/CACHE/chr10test34.pickle
564.2739849984646
../data/wg20220316/chr10test/tmp/CACHE/chr10test33.pickle
287.9965378642082
../data/wg20220316/chr10test/tmp/CACHE/chr10test32.pickle
230.90928138792515
../data/wg20220316/chr10test/tmp/CACHE/chr10test31.pickle
274.10094122588634
../data/wg20

In [ ]:
def run_gene_lods(file,fam,rho=0,cutoff=None):
    with open(file+'.pickle', 'rb') as handle:
        genes = pickle.load(handle)
    gene_variants,gene_fam_haps = format_haps_bunch(genes,fam)
    if cutoff is not None:
        for f,variants in gene_variants.items():
            gene_fam_haps[f]=gene_fam_haps[f].loc[:,[True]*6+list(np.repeat((variants.freqs>cutoff)[variants.uniq],2))]
    res = parallel_lods(gene_fam_haps.values(),rho)
    smy_res = sum_variant_lods(res)
    with open(file+'cutoff'+str(cutoff)+'_rho'+str(rho)+'.result','wb') as handle:
        pickle.dump(smy_res, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
def format_haps_by_genes(file,fam,cutoff=None):
    with open(file+'.pickle', 'rb') as handle:
        genes = pickle.load(handle)
    gene_variants,gene_fam_haps = format_haps_bunch(genes,fam)
    if cutoff is not None:
        for f,variants in gene_variants.items():
            gene_fam_haps[f]=gene_fam_haps[f].loc[:,[True]*6+list(np.repeat((variants.freqs>cutoff)[variants.uniq],2))]
    with open(file+'cutoff'+str(cutoff)+'.input','wb') as handle:
        pickle.dump([gene_variants,gene_fam_haps], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
file='../data/wg20220316/chr22test/tmp/CACHE/chr22test24cutoff0.05.input'
with open(file, 'rb') as handle:
    gene_variants,gene_fam_haps = pickle.load(handle)
res = parallel_lods(gene_fam_haps.values(),np.arange(0,0.5,0.05))
with open(file[:-6]+'.lods','wb') as handle:
    pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)

610.6680277064443


In [ ]:
sos run nbs/seqlink_sos.ipynb lods --cwd data/wg20220316 --fam_path data/new_trim_ped_famless17_no:xx.fam --chrom 22 -j 1

In [ ]:
format_haps_by_genes('../data/wg20220311/chr19test/CACHE/chr19test43',fam17_d,cutoff=0.05)

In [ ]:
run_gene_lods('../data/wg20220311/chr19test/CACHE/chr19test44',fam17_d,rho=0.05,cutoff=0.05)

81.34782417491078


In [ ]:
with open('../data/wg20220316/fam17_vcf.pickle', 'rb') as handle:
    fam17_vcf = pickle.load(handle)

In [ ]:
with open('../data/wg20220421/chr22test/tmp/CACHE/chr22test0.pickle', 'rb') as handle:
    genes = pickle.load(handle)

In [ ]:
gene_variants,gene_fam_haps = format_haps_bunch(genes,fam17_d,fam17_vcf,cutoff=0.01,haplotype=False)

In [ ]:
with open('../data/wg20220316/chr22test/tmp/CACHE/chr22test24cutoff0.05.input', 'rb') as handle:
    gene_variants,gene_fam_haps = pickle.load(handle)

In [ ]:
tmp=gene_fam_haps['1007']

In [ ]:
for x,y in zip(gene_fam_haps.values(),[gene_variants[k] for k in gene_fam_haps.keys()]):
    if (x.shape[1]-6)/2!=sum(y.uniq):
        print(x.fid[0])

1007
1010
1033
1036
10J_103
10J_109
10J_112
10J_119
10J_121
10J_123
10J_125
10J_128
10J_131
10J_82
10J_89
10R_R101
10R_R105
10R_R107
10R_R109
10R_R111
10R_R16
10R_R38
10R_R4
10R_R48
10R_R54
10R_R56
10R_R66
10R_R78
10R_R87
10R_R99
1137
1154
1177
1197
11_13
11_15
11_19
1214
1227
1241
1252
127
1277
1279
1317
135
1359
1403
1438
1462
1484
14_1130
14_1658
14_1943
14_TJ
150
1517
156
1582
1584
15_10052
15_1121
15_12091
15_2131
15_3111
15_4041
15_6051
15_7051
15_7053
15_8053
15_9044
15_9052
15_9053
161
164
168
1697
1713
173
1731
174
175
1755
1769
1783
17_13
17_15
17_17
17_25
17_27
17_5
1819
1841
1853
1858
1880
191
1917
1926
1928
195
1960
1963
1966
197
1989
199
1999
19_L0010
19_L0027
19_L0028
19_L0030
19_L008
2
20
2004
203
204
2045
205
2075
210
211
2128
2129
216
2193
21_ND128
21_ND386
221
222
223
224
22_1
22_70
242
246
248
251
25_22
25_30
25_37
25_41
25_42
25_5
25_56
25_57
25_61
25_62
25_66
25_67
25_68
25_69
25_70
25_71
25_72
25_73
25_74
25_75
25_76
25_78
25_80
25_81
263
264
26_AMC
26_ANR
26_ARH

In [ ]:
afreqs = []
for k in gene_fam_haps.keys():
    variants= gene_variants[k]
    variants=variants.freqs[variants.uniq]
    #variants=variants[variants>0.05]
    afreqs.append(list(variants.round(decimals=3)))

In [ ]:
tmp=[]
for i in range(10):
    try:
        hap=gene_fam_haps[list(gene_fam_haps.keys())[i]]
        frq=afreqs[i]
        tmp.append(calculate_ped_lod(hap,frq,np.arange(0,0.5,0.05)))
    except:
        print(i)

In [ ]:
res = parallel_lods(gene_fam_haps,afreqs,np.arange(0,0.5,0.05))

1834.5320355594158


In [ ]:
len(res)

479

In [ ]:
0<1<2

True

In [ ]:
genes['APOE']['predata']['1007'][2]

array([['1007', '1007_1', '2:', '1:', 'A1,2:', '1:'],
       ['1007', '1007_1', '2:', '2:', '1:', '2:'],
       ['1007', '1007_2', '1:', '1:', 'A2,1:', '1:'],
       ['1007', '1007_2', '2:', '2:', '1:', '2:'],
       ['1007', '1007_40', '2:', '1:', 'A1,2:', '1:'],
       ['1007', '1007_40', '?:', '?:', '?:', '?:'],
       ['1007', '1007_99', '2:', '1/', 'A2,1|', '1\\'],
       ['1007', '1007_99', '2:', '1/', 'A1,2|', '2\\'],
       ['1007', '1007_5', '2:', '2|', '1|', '2|'],
       ['1007', '1007_5', '2:', '2|', '1|', '2|'],
       ['1007', '1007_3', '1:', '1|', 'A2,1|', '1|'],
       ['1007', '1007_3', '2:', '1|', 'A1,2|', '1|'],
       ['1007', '1007_6', '1:', '1|', 'A2,1|', '1|'],
       ['1007', '1007_6', '2:', '1|', 'A1,2|', '1|'],
       ['1007', '1007_4', '1:', '1|', 'A2,1|', '1|'],
       ['1007', '1007_4', '2:', '1|', 'A1,2|', '1|'],
       ['1007', '1007_39', '1:', '1|', 'A2,1|', '1|'],
       ['1007', '1007_39', '2:', '1|', 'A1,2|', '1|']], dtype='<U7')

### Functions of heterogeneity

In [ ]:
with open('../data/wg20220316/chr22test/tmp/CACHE/chr22test24cutoff0.05.lods', 'rb') as handle:
    res = pickle.load(handle)

In [ ]:
res[0]

,LOD0.0,LOD0.05,LOD0.1,LOD0.15,LOD0.2,LOD0.25,LOD0.3,LOD0.35,LOD0.4,LOD0.45
chr22:50343347:G:C_A0,-0.381789,-0.207810,-0.120815,-0.072131,-0.043658,-0.026540,-0.015871,-0.008874,-0.004121,-0.001113
chr22:50343681:G:GT_A0,-0.272838,-0.186707,-0.128434,-0.087482,-0.058191,-0.037161,-0.022198,-0.011827,-0.005054,-0.001234
chr22:50346343:C:T_A0,-1.408093,-0.732202,-0.472576,-0.315381,-0.208638,-0.133119,-0.079395,-0.042078,-0.017782,-0.004262
chr22:50346737:G:A_A0,-0.442093,-0.391146,-0.317935,-0.242735,-0.175166,-0.118663,-0.073878,-0.040392,-0.017448,-0.004240
chr22:50347289:C:T_A0,-0.272838,-0.186707,-0.128434,-0.087482,-0.058191,-0.037161,-0.022198,-0.011827,-0.005054,-0.001234
...,...,...,...,...,...,...,...,...,...,...
chr22:50796715:A:AACAC_A0,-0.257678,-0.244179,-0.213902,-0.175194,-0.134540,-0.096239,-0.062871,-0.035904,-0.016150,-0.004080
chr22:50796715:AAC:A_A0,-0.296969,-0.276525,-0.237668,-0.191120,-0.144346,-0.101737,-0.065597,-0.037021,-0.016475,-0.004120
chr22:50797551:G:A_A0,-0.296969,-0.276525,-0.237668,-0.191120,-0.144346,-0.101737,-0.065597,-0.037021,-0.016475,-0.004120
chr22:50797585:A:AT_A0,-0.296969,-0.276525,-0.237668,-0.191120,-0.144346,-0.101737,-0.065597,-0.037021,-0.016475,-0.004120


In [ ]:
res=list(res)

In [ ]:
variants = sorted(list(set().union(*[i.index for i in res])))

In [ ]:
def format_fam_lods(res):
    new_res,variants=[],[]
    for i in res:
        new_res.append(i)
        variants.append(i.index)
    variants = list(set().union(*variants))
    cutoff = len(new_res)//10
    var_res={}
    for v in variants:
        varlods = [r.loc[v] for r in res if v in r.index]
        if len(varlods)>cutoff:
            var_res[v]=pd.concat(varlods,axis=1)
    return var_res

In [ ]:
start = time.perf_counter()
var_res=format_fam_lods(res)
print(time.perf_counter()-start)

474.6678010523319


In [ ]:
list(var_res.keys())[:10]

['chr22:50391432:CA:C_A0',
 'chr22:50768026:G:T_A0',
 'chr22:50743074:G:C_A0',
 'chr22:50697023:G:C_A0',
 'chr22:50402606:C:T_A0',
 'chr22:50514141:A:G_A0',
 'chr22:50435332:A:G_A0',
 'chr22:50588698:T:TA_A0',
 'chr22:50723631:G:A_A0',
 'chr22:50382519:CA:C_A0']

In [ ]:
def hlod_fun(Li, sign=1):
    def _fun(alpha):
        return sign * sum(np.log10(alpha*np.power(10, Li) + 1 - alpha))
    return _fun

In [ ]:
start = time.perf_counter()
var_sovs=[]
for var,res in var_res.items():
    for theta in res.index:
        sov = minimize_scalar(hlod_fun(list(res.loc[theta]), -1), bounds=(0,1), method='bounded', options={'xatol':1e-8})
        var_sovs.append([var,theta,sov.x,-sov.fun])
print(time.perf_counter()-start)

NameError: name 'minimize_scalar' is not defined

In [ ]:
def min_hlod_func(res):
    var_sovs=[]
    for theta in res.index:
        sov = minimize_scalar(hlod_fun(list(res.loc[theta]), -1), bounds=(0,1), method='bounded', options={'xatol':1e-8})
        var_sovs.append([var,theta,sov.x,-sov.fun])
    return var,theta,sov.x,-sov.fun

In [ ]:
start = time.perf_counter()
results1=[]
with ProcessPoolExecutor(max_workers = 10) as executor:
    results = executor.map(min_hlod_func,var_res.values())
#for i in results:
#    results1.append(i)     
print(time.perf_counter()-start)

2.828973636031151


In [ ]:
for i in results:
    pass

In [ ]:
results=list(results)

### Pipeline of heterogeneity

In [ ]:
from scipy.optimize import minimize_scalar

In [ ]:
lod_files=glob.glob('../data/wg20220316/chr22test/tmp/CACHE/chr22test*cutoff0.05.lods')

In [ ]:
for file in lod_files:
    print(file[:-5])
    with open(file, 'rb') as handle:
        res = pickle.load(handle)
    var_res=format_fam_lods(res)
    start = time.perf_counter()
    var_sovs,best_sovs=[],[]
    for var,res in var_res.items():
        best_sov=[var,'LOD0.5',0,0]
        for theta in res.index:
            sov = minimize_scalar(hlod_fun(list(res.loc[theta]), -1), bounds=(0,1), method='bounded', options={'xatol':1e-8})
            var_sov=[var,theta,sov.x,-sov.fun]
            var_sovs.append(var_sov)
            if best_sov[3]<var_sov[3]: 
                best_sov=var_sov
        best_sovs.append(best_sov)
    print(time.perf_counter()-start)
    var_sovs=pd.DataFrame(var_sovs)
    best_sovs=pd.DataFrame(best_sovs)
    with open(file[:-5]+'.hlods','wb') as handle:
        pickle.dump(var_sovs, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(file[:-5]+'.besthlod','wb') as handle:
        pickle.dump(best_sovs, handle, protocol=pickle.HIGHEST_PROTOCOL)

../data/wg20220316/chr22test/tmp/CACHE/chr22test23cutoff0.05
232.4731414541602
../data/wg20220316/chr22test/tmp/CACHE/chr22test22cutoff0.05
128.7288079187274
../data/wg20220316/chr22test/tmp/CACHE/chr22test21cutoff0.05
767.982909552753
../data/wg20220316/chr22test/tmp/CACHE/chr22test20cutoff0.05
94.22814803570509
../data/wg20220316/chr22test/tmp/CACHE/chr22test19cutoff0.05
39.74833147972822
../data/wg20220316/chr22test/tmp/CACHE/chr22test18cutoff0.05
31.876558922231197
../data/wg20220316/chr22test/tmp/CACHE/chr22test17cutoff0.05
53.270271331071854
../data/wg20220316/chr22test/tmp/CACHE/chr22test16cutoff0.05
48.72832177579403
../data/wg20220316/chr22test/tmp/CACHE/chr22test15cutoff0.05
60.962094478309155
../data/wg20220316/chr22test/tmp/CACHE/chr22test14cutoff0.05
57.58952667564154
../data/wg20220316/chr22test/tmp/CACHE/chr22test13cutoff0.05
497.4589391872287
../data/wg20220316/chr22test/tmp/CACHE/chr22test12cutoff0.05
871.1246994510293
../data/wg20220316/chr22test/tmp/CACHE/chr22test11

In [ ]:
lod_files=glob.glob('../data/wg20220316/chr22test/tmp/CACHE/chr22test*cutoff0.05unimputed.lods')

In [ ]:
for file in lod_files:
    print(file[:-5])
    with open(file, 'rb') as handle:
        res = pickle.load(handle)
    var_res=format_fam_lods(res)
    start = time.perf_counter()
    var_sovs,best_sovs=[],[]
    for var,res in var_res.items():
        best_sov=[var,'LOD0.5',0,0]
        for theta in res.index:
            sov = minimize_scalar(hlod_fun(list(res.loc[theta]), -1), bounds=(0,1), method='bounded', options={'xatol':1e-8})
            var_sov=[var,theta,sov.x,-sov.fun]
            var_sovs.append(var_sov)
            if best_sov[3]<var_sov[3]: 
                best_sov=var_sov
        best_sovs.append(best_sov)
    print(time.perf_counter()-start)
    var_sovs=pd.DataFrame(var_sovs)
    best_sovs=pd.DataFrame(best_sovs)
    with open(file[:-5]+'.hlods','wb') as handle:
        pickle.dump(var_sovs, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(file[:-5]+'.besthlod','wb') as handle:
        pickle.dump(best_sovs, handle, protocol=pickle.HIGHEST_PROTOCOL)

../data/wg20220316/chr22test/tmp/CACHE/chr22test24cutoff0.05unimputed
44.67064615339041
../data/wg20220316/chr22test/tmp/CACHE/chr22test23cutoff0.05unimputed
205.38292413949966
../data/wg20220316/chr22test/tmp/CACHE/chr22test22cutoff0.05unimputed
168.32762810587883
../data/wg20220316/chr22test/tmp/CACHE/chr22test21cutoff0.05unimputed
201.9195803552866
../data/wg20220316/chr22test/tmp/CACHE/chr22test20cutoff0.05unimputed
160.12764471769333
../data/wg20220316/chr22test/tmp/CACHE/chr22test19cutoff0.05unimputed
67.1177901700139
../data/wg20220316/chr22test/tmp/CACHE/chr22test18cutoff0.05unimputed
53.2965392023325
../data/wg20220316/chr22test/tmp/CACHE/chr22test17cutoff0.05unimputed
86.53747600317001
../data/wg20220316/chr22test/tmp/CACHE/chr22test16cutoff0.05unimputed
50.05957846343517
../data/wg20220316/chr22test/tmp/CACHE/chr22test15cutoff0.05unimputed
56.389251589775085
../data/wg20220316/chr22test/tmp/CACHE/chr22test14cutoff0.05unimputed
80.27903558313847
../data/wg20220316/chr22test/t

In [ ]:
with open('../data/wg20220316/chr22test/tmp/CACHE/chr22test24cutoff0.05'+'.besthlod','rb') as handle:
    best_sovs=pickle.load(handle)

In [ ]:
best_sovs

,0,1,2,3
0,chr22:50391432:CA:C_A0,LOD0.3,0.052296,0.002507
1,chr22:50768026:G:T_A0,LOD0.3,0.841278,1.863049
2,chr22:50743074:G:C_A0,LOD0.3,0.175603,0.173724
3,chr22:50697023:G:C_A0,LOD0.25,0.308812,1.243402
4,chr22:50402606:C:T_A0,LOD0.3,0.148316,0.138290
...,...,...,...,...
1195,chr22:50490618:CA:C_A0,LOD0.2,0.754465,1.728065
1196,chr22:50504803:G:A_A0,LOD0.25,0.167891,0.329213
1197,chr22:50794884:A:G_A0,LOD0.35,0.804642,0.849943
1198,chr22:50572812:C:T_A0,LOD0.3,1.000000,0.815049


### Pipeline of linkage analysis (without haplotype imputation)

In [ ]:
ped_vcf=pd.read_csv('../data/new_trim_ped.csv')
ped_vcf.index=list(ped_vcf.iid)
fam17_vcf={}
for k,v in fam17_d.items():
    fam17_vcf[k]=ped_vcf.vcf[v.index]

In [ ]:
with open('../data/wg20220316/fam17_vcf.pickle','wb') as handle:
    pickle.dump(fam17_vcf, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
file='../data/wg20220316/chr22test/tmp/CACHE/chr22test24cutoff0.05.input'
with open(file, 'rb') as handle:
    gene_variants,gene_fam_haps = pickle.load(handle)

In [ ]:
res = parallel_lods(gene_fam_haps.values(),np.arange(0,0.5,0.05))
with open(file[:-6]+'.lods','wb') as handle:
    pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)

610.6680277064443


In [ ]:
hap=gene_fam_haps['1007']

In [ ]:
def unimput_haps():
    pass

In [ ]:
hap.shape

(9, 1684)

In [ ]:
for k,hap in gene_fam_haps.items():
    hap.loc[~fam17_vcf[k],[False]*6+[True]*(hap.shape[1]-6)]=0   

In [ ]:
ped_vcf.vcf[gene_fam_haps['1007'].iid]

1007_39    False
1007_99    False
1007_3     False
1007_5     False
1007_40     True
1007_6      True
1007_1      True
1007_2      True
1007_4      True
Name: vcf, dtype: bool

pseudomarker -p test_f10.ped -m test_f10.map --dom

# Merlin to linkage

In [ ]:
cmap=pd.read_csv('../data/wg20220316/chr22test/MERLIN/chr22test.chr22.map',sep='\t')

In [ ]:
new_map=cmap.iloc[:,[0,2,1]]

In [ ]:
new_map.columns = ['Chromosome','Haldane','Name']

In [ ]:
new_map.to_csv('../data/wg20220316/chr22test/MERLIN/chr22test.chr22_new.map',header=True,index=False,sep='\t')

In [ ]:
new_map

,Chromosome,Haldane,Name
0,22,0.000000,DUXAP8
1,22,0.000000,BMS1P22@3
2,22,0.000000,"BMS1P17@3,BMS1P18@3"
3,22,0.000000,PSLNR
4,22,0.000000,POTEH
...,...,...,...
584,22,78.751192,ALG12
585,22,78.788843,CRELD2
586,22,78.895162,PIM3
587,22,79.101809,IL17REL


In [ ]:
cped = pd.read_csv('../data/wg20220316/chr22test/MERLIN/chr22test.chr22.ped',sep='\t',header=None)

/tmp/2295440.1.high_mem.q/ipykernel_26744/410291780.py:1: DtypeWarning: Columns (0,66,67,148,149,158,159,162,175,180,181,226,227,236,237,244,248,249,250,251,266,267,417,470,471,500,501,524,525,592,593,656,658,659,860,861,864,866,918,919,968,969,1106,1107,1122,1123,1124,1125,1126,1127,1168,1169) have mixed types. Specify dtype option on import or set low_memory=False.
  cped = pd.read_csv('../data/wg20220316/chr22test/MERLIN/chr22test.chr22.ped',sep='\t',header=None)


In [ ]:
cped.shape

(3899, 1184)

In [ ]:
for i in range(0,cped.shape[1]-6,2):
    tmp0 = cped.iloc[:,6+i]
    tmp1 = cped.iloc[:,7+i]
    ind = (tmp0==0) | (tmp1==0)
    tmp0[ind]=0
    tmp1[ind]=0
    tmp0[tmp0.astype(int)>2]=2
    tmp1[tmp1.astype(int)>2]=2

/tmp/2296001.1.high_mem.q/ipykernel_31617/590733060.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp0[ind]=0
/tmp/2296001.1.high_mem.q/ipykernel_31617/590733060.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1[ind]=0
/tmp/2296001.1.high_mem.q/ipykernel_31617/590733060.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp0[tmp0.astype(int)>2]=2
/tmp/2296001.1.high_mem.q/ipykernel_31617/590733060.py:8: SettingWithCopyWar

In [ ]:
cped[5]=cped[5].replace(-9,0)

In [ ]:
cped.index = list(cped[1])

In [ ]:
cped=cped.sort_index()

In [ ]:
cped.to_csv('../data/wg20220316/chr22test/MERLIN/chr22test.chr22_new.ped',header=False,index=False,sep='\t')

In [ ]:
cped.iloc[:,:26].to_csv('../data/wg20220316/chr22test/MERLIN/chr22test.chr22_new_f10.ped',header=False, index=False,sep='\t')

In [ ]:
new_map[:10].to_csv('../data/wg20220316/chr22test/MERLIN/chr22test.chr22_new_f10.map',header=True,index=False,sep='\t')

In [ ]:
cped

,0,1,2,3,4,5,6,7,8,9,...,1174,1175,1176,1177,1178,1179,1180,1181,1182,1183
1007_1,1007,1007_1,0,0,1,0,1,2,1,2,...,0,0,0,0,1,2,2,1,0,0
1007_2,1007,1007_2,0,0,2,0,1,2,0,0,...,2,2,2,1,0,0,0,0,1,2
1007_3,1007,1007_3,1007_1,1007_2,1,2,0,0,0,0,...,2,1,1,1,0,0,2,2,0,0
1007_39,1007,1007_39,1007_40,1007_4,1,1,2,2,2,1,...,2,2,1,2,2,2,2,2,2,2
1007_4,1007,1007_4,1007_1,1007_2,2,0,1,2,0,0,...,0,0,0,0,1,1,1,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
989_99,989,989_99,989_1,989_2,2,2,0,0,2,2,...,2,2,1,1,0,0,2,2,2,2
990_1,990,990_1,0,0,1,0,0,0,2,2,...,1,2,1,1,2,1,2,2,0,0
990_12,990,990_12,990_1,990_2,2,2,0,0,0,0,...,2,2,2,2,2,1,0,0,0,0
990_2,990,990_2,0,0,2,0,2,2,2,1,...,1,1,0,0,0,0,2,1,0,0


## Run paramlink2 on CHP markers

In [ ]:
cped = pd.read_csv('../data/wg20220316/chr22test/MERLIN/chr22test.chr22.ped',sep='\t',header=None)

/tmp/2296001.1.high_mem.q/ipykernel_31617/410291780.py:1: DtypeWarning: Columns (0,66,67,148,149,158,159,162,175,180,181,226,227,236,237,244,248,249,250,251,266,267,417,470,471,500,501,524,525,592,593,656,658,659,860,861,864,866,918,919,968,969,1106,1107,1122,1123,1124,1125,1126,1127,1168,1169) have mixed types. Specify dtype option on import or set low_memory=False.
  cped = pd.read_csv('../data/wg20220316/chr22test/MERLIN/chr22test.chr22.ped',sep='\t',header=None)


In [ ]:
cped=cped.replace('?',0)

In [ ]:
cped = pd.concat([cped.iloc[:,:4].astype(str),cped.iloc[:,4:].astype(int)],axis=1)

In [ ]:
cped.index = list(cped[1])

In [ ]:
cped=cped.sort_index()

In [ ]:
cped[5]=cped[5].replace(-9,0)

In [ ]:
tmp = cped.iloc[:,6:]

In [ ]:
tmp[tmp>2]=2

In [ ]:
cped = pd.concat([cped.iloc[:,:6],tmp],axis=1)

In [ ]:
cped_d={}
for i in cped[0].unique():
    cped_d[i]=cped[cped[0]==i]

In [ ]:
calculate_ped_lod(cped_d['1137'])

,MARKER,LOD
1,6,NaN
2,8,0.009661
3,10,NaN
4,12,0.009661
5,14,0.283737
...,...,...
585,1174,NaN
586,1176,0.283529
587,1178,-0.005014
588,1180,0.000000


In [ ]:
cped_res = parallel_lods(cped_d.values())

50.33882123604417


In [ ]:
cmap

,CHROMOSOME,MARKER,Unnamed: 2,POSITION,FEMALE_POSITION,MALE_POSITION
0,22,DUXAP8,0.000000,0.000000,0.000000,NaN
1,22,BMS1P22@3,0.000000,0.000000,0.000000,NaN
2,22,"BMS1P17@3,BMS1P18@3",0.000000,0.000000,0.000000,NaN
3,22,PSLNR,0.000000,0.000000,0.000000,NaN
4,22,POTEH,0.000000,0.000000,0.000000,NaN
...,...,...,...,...,...,...
584,22,ALG12,78.751192,68.840233,89.808657,NaN
585,22,CRELD2,78.788843,68.907445,89.814048,NaN
586,22,PIM3,78.895162,69.096823,89.829375,NaN
587,22,IL17REL,79.101809,69.467435,89.859470,NaN


In [ ]:
variants = {}
for lod in cped_res:
    for m,l in zip(lod['MARKER'],lod['LOD']):
        if pd.isna(l):
            continue
        if m in variants.keys():
            variants[m] += l
        else:
            variants[m] = l
#variants=pd.DataFrame(variants)

ValueError: If using all scalar values, you must pass an index

In [ ]:
variants

In [ ]:
cped_d['1007'].to_csv('../data/wg20220316/chr22test/MERLIN/chr22test.chr22_new_1007.ped',header=False,index=False,sep='\t')